In [1]:
import pandas as pd

# Specify the file path
file_path = r"C:\Users\VAMSI\Desktop\project\Bird_Monitoring_Data_GRASSLAND.XLSX"

# Read the Excel file with multiple sheets
excel_data = pd.ExcelFile(file_path)

# Get all sheet names
sheet_names_GRASSLAND = excel_data.sheet_names

# Read data from all sheets into a dictionary
sheets_dict_grass = {}
for sheet in sheet_names_GRASSLAND:
    df = excel_data.parse(sheet)
    # Drop completely empty columns before processing
    df = df.dropna(axis=1, how='all')
    sheets_dict_grass[sheet] = df

# Combine all sheets into one DataFrame
combined_df_grass = pd.concat(
    [df.assign(Sheet=sheet_name) for sheet_name, df in sheets_dict_grass.items()],
    ignore_index=True
)

# Drop the 'Sheet' column if you don't need it
combined_df_GRASSLAND = combined_df_grass.drop(columns=['Sheet'])


In [2]:
combined_df_GRASSLAND.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8531 entries, 0 to 8530
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Admin_Unit_Code              8531 non-null   object        
 1   Plot_Name                    8531 non-null   object        
 2   Location_Type                8531 non-null   object        
 3   Year                         8531 non-null   float64       
 4   Date                         8531 non-null   datetime64[ns]
 5   Start_Time                   8531 non-null   object        
 6   End_Time                     8531 non-null   object        
 7   Observer                     8531 non-null   object        
 8   Visit                        8531 non-null   float64       
 9   Interval_Length              8531 non-null   object        
 10  ID_Method                    8530 non-null   object        
 11  Distance                     7137 non-null 

In [3]:
combined_df_GRASSLAND ['Date'] = pd.to_datetime(combined_df_GRASSLAND ['Date'], errors='coerce')

def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

# Apply the function to the datetime column
combined_df_GRASSLAND ['Season'] = combined_df_GRASSLAND ['Date'].dt.month.apply(get_season)

In [4]:
#changing data type of visit and year from float to int
combined_df_GRASSLAND['Visit'] = combined_df_GRASSLAND['Visit'].round(0).astype(int)

In [5]:
#check for duplicates and print the total dupes
no_of_dupes=combined_df_GRASSLAND.duplicated().sum()
no_of_dupes

1705

In [6]:
combined_df_GRASSLAND = combined_df_GRASSLAND.drop_duplicates()

In [7]:
combined_df_GRASSLAND.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6826 entries, 0 to 8521
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Admin_Unit_Code              6826 non-null   object        
 1   Plot_Name                    6826 non-null   object        
 2   Location_Type                6826 non-null   object        
 3   Year                         6826 non-null   float64       
 4   Date                         6826 non-null   datetime64[ns]
 5   Start_Time                   6826 non-null   object        
 6   End_Time                     6826 non-null   object        
 7   Observer                     6826 non-null   object        
 8   Visit                        6826 non-null   int32         
 9   Interval_Length              6826 non-null   object        
 10  ID_Method                    6825 non-null   object        
 11  Distance                     6229 non-null   obj

In [8]:
#filling the null value of id method with mode of 'ID_Method ' column
combined_df_GRASSLAND['ID_Method']= combined_df_GRASSLAND['ID_Method'].fillna(combined_df_GRASSLAND['ID_Method'].mode()[0])

In [9]:
#filling the null value of distance with mode of 'distance' column based on proportions
import numpy as np

# Step 1: Get frequency distribution of non-null values
value_counts = combined_df_GRASSLAND['Distance'].value_counts(normalize=True)  # Gets probabilities

# Step 2: Randomly sample missing values based on their likelihood
null_mask = combined_df_GRASSLAND['Distance'].isna()
combined_df_GRASSLAND.loc[null_mask, 'Distance'] = np.random.choice(
    value_counts.index, 
    size=null_mask.sum(), 
    p=value_counts.values
)

In [10]:
# Remove spaces and alphabets next
combined_df_GRASSLAND['Distance'] = combined_df_GRASSLAND['Distance'].replace(r'\s+', '', regex=True)  
combined_df_GRASSLAND['Distance'] = combined_df_GRASSLAND['Distance'].replace(r'[a-zA-Z]', '', regex=True)

In [11]:
combined_df_GRASSLAND.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6826 entries, 0 to 8521
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Admin_Unit_Code              6826 non-null   object        
 1   Plot_Name                    6826 non-null   object        
 2   Location_Type                6826 non-null   object        
 3   Year                         6826 non-null   float64       
 4   Date                         6826 non-null   datetime64[ns]
 5   Start_Time                   6826 non-null   object        
 6   End_Time                     6826 non-null   object        
 7   Observer                     6826 non-null   object        
 8   Visit                        6826 non-null   int32         
 9   Interval_Length              6826 non-null   object        
 10  ID_Method                    6826 non-null   object        
 11  Distance                     6826 non-null   obj

In [12]:
number_0f_unique_AcceptedTSN =combined_df_GRASSLAND['AcceptedTSN'].nunique()
print(number_0f_unique_AcceptedTSN)

number_0f_unique_Scientific_Name =combined_df_GRASSLAND['Scientific_Name'].nunique()
print(number_0f_unique_Scientific_Name)

number_0f_unique_TaxonCode  =combined_df_GRASSLAND['TaxonCode'].nunique()
print(number_0f_unique_TaxonCode)

number_0f_unique_AOU_Code  = combined_df_GRASSLAND['AOU_Code'].nunique()
print(number_0f_unique_AOU_Code)

106
107
105
107


In [13]:
# filling the AcceptedTSN with 997805 and changing all float datatypes to int 
combined_df_GRASSLAND['AcceptedTSN'] = combined_df_GRASSLAND['AcceptedTSN'].fillna(997805)
combined_df_GRASSLAND['AcceptedTSN'] = combined_df_GRASSLAND['AcceptedTSN'].round(0).astype(int)

In [14]:
#filling the TaxonCode with 0 
combined_df_GRASSLAND['TaxonCode'] = combined_df_GRASSLAND['TaxonCode'].fillna(0)

In [15]:
combined_df_GRASSLAND.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6826 entries, 0 to 8521
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Admin_Unit_Code              6826 non-null   object        
 1   Plot_Name                    6826 non-null   object        
 2   Location_Type                6826 non-null   object        
 3   Year                         6826 non-null   float64       
 4   Date                         6826 non-null   datetime64[ns]
 5   Start_Time                   6826 non-null   object        
 6   End_Time                     6826 non-null   object        
 7   Observer                     6826 non-null   object        
 8   Visit                        6826 non-null   int32         
 9   Interval_Length              6826 non-null   object        
 10  ID_Method                    6826 non-null   object        
 11  Distance                     6826 non-null   obj

In [16]:
#check for duplicates and print the total dupes
no_of_dupes=combined_df_GRASSLAND.duplicated().sum()
no_of_dupes

0

In [17]:
# writing the cleaned dataframe into csv file
combined_df_GRASSLAND.to_csv("_Cleaned_GRASSLAND_df_.csv",index=False)